<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Main-part-of-notebook-with-functional-approach-to-present-and-manipulate-data" data-toc-modified-id="Main-part-of-notebook-with-functional-approach-to-present-and-manipulate-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Main part of notebook with functional approach to present and manipulate data</a></span></li><li><span><a href="#Supplement" data-toc-modified-id="Supplement-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Supplement</a></span></li></ul></div>

In [1]:
import os, re, csv, os
import random
from glob import glob
from datetime import date, datetime, timedelta
import webcolors
import numpy as np
import pandas as pd
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go
import plotly.tools as tls
from plotly import subplots
import cufflinks as cf
cf.go_offline()
import plotly.express as px

#### Main part of notebook with functional approach to present and manipulate data

In [2]:
def get_file(template):
    """Function checks if there is only 1 file for given template
    Return: path of file in /Data/wind_csv/ folder"""
    # glob function creates a list of files appropriate with template
    filenames = glob(template)
    if len(filenames) == 1:
        return filenames[0]
    elif len(filenames) == 0:
        return
    else:
        raise Exception(f'There are more than one files in /Data/wind_csv/ folder - leave only 1 and repeat!')
    
def get_raw_file_path(year, month):
    """Return: path template to the raw(source) file for given year and month"""
    return get_file(f'./Data/wind_csv/{year}/PL_GEN_WIATR_{year}{month:02}*.csv')

def get_raw_files_list(year):
    """
    Return: 2 lists: first one for downloaded from PSE webpage files, second one for months numbers of appropriate
    files. When there are no files downloaded or duplicates - error messages are generated.
    """
    raw_files = []
    month_numbers = []
    for month_num in range(1, 13):
        file = get_raw_file_path(year, month_num)
        if file:
            raw_files.append(file)
            month_numbers.append(month_num)
    if raw_files == []:
        return
    
    return raw_files, month_numbers

In [3]:
def transformed_file_path(year, month):
    if not os.path.exists(f'./Data/wind_ready/{year}/'):
        os.makedirs(f'./Data/wind_ready/{year}')   
    return f'./Data/wind_ready/{year}/{year}{month:02}.csv'

def get_transformed_file(year, month):
    return get_file(transformed_file_path(year, month))

def files_list(year):
    """
    Return: List of transformed 'csv' files for given year or message when the data is lacking.
    """
    ready_list = sorted(glob(f'./Data/wind_ready/{year}/*.csv'))
    raw_files = get_raw_files_list(year)
    if  len(ready_list) == 0:
        if raw_files is None:
            if year <= datetime.now().year and year >= 2012:
                print(f'There is no data for {year}. You should try to download it from PSE webpage.')
            else:
                print(f'Data for {year} year is not available.')
            return
        
        for month_number in raw_files[1]:
            save_clean_data(year, month_number)
        ready_list = sorted(glob(f'./Data/wind_ready/{year}/*.csv'))
    month_numbers = raw_files[1]
    month_numbers.append('all')
    return ready_list, month_numbers                

In [4]:
def save_clean_data(year, month_num):
    df = pd.read_csv(get_raw_file_path(year, month_num),
                     encoding='iso 8859-1',
                     sep=';',
                     skiprows=[0],
                     usecols=[0, 1, 2],
                     names=['Date', 'Time', 'Total_Wind_Power(MWh)'],
                     index_col='Date',
                    converters={1: lambda x:x.replace('2A', '2'),
                               2: lambda x:x.replace(',', '.')})

    df.to_csv(transformed_file_path(year, month_num), header=True)
    
def get_clean_data(year, month):
    if not os.path.exists(f'./Data/wind_ready/{year}/{year}{month}.csv'):
        save_clean_data(year, month)
    return pd.read_csv(get_transformed_file(year, month))

In [5]:
def wind_hourly(year, month_num):
    """
    Input: function takes the year and the number of month in range between 1 and number of months for given year 
    Return: dataframe either for month's wind power generation or for all months - to use for visualization, analysis,
    modelling.
    """
    months_numbers = files_list(year)[1]
    
    # preparing dataframe either for all months in year or for only one month
    if month_num == 'all':
        df_all = [get_clean_data(year, month_num) for month_num in months_numbers[:-1]]
        df = pd.concat(df_all)
    elif month_num in months_numbers:
        df = get_clean_data(year, month_num)
    else:
        print(f"Data for {month_name(month_num)} of {year} year is not available.")
        return
    
    df['Date'] = pd.to_datetime(df['Date'].astype('str'))
    df.set_index('Date', inplace=True)
    
    return df

In [6]:
def wind_daily(year, month_num):
    """
    Return: dataframe of wind generation where indexing is by day values not by hours
    """
    if month_num not in files_list(year)[1]:
        print("Data for that month is not available or wrong parameter was given for month number.")
        return
    # resampled data from hours to days
    df_days = wind_hourly(year, month_num).resample('D').sum().iloc[:,[1]]
    df_days.rename(columns={'Total_Wind_Power(MWh)':'Wind_Daily(MWh)'}, inplace=True)
    
    return df_days

In [7]:
def month_name(month_num):
    """
    Function month_name() returns name of month  to use it in formatting strings for plotting labels. 
    """
    return date(1990, int(month_num), 1).strftime('%B')

def month_names(year):
    return [month_name(month_num) for month_num in files_list(year)[1][:-1]]

def years_list():
    y_list = os.listdir('./Data/wind_csv')
    if y_list[0] == 'ipynb':
        return y_list[1:]
    else:
        return y_list
    
def incorrect_year(year):
    if str(year) not in years_list():
        print(f"No data for a given year: {year}")
        return
    else:
        return 'OK'

In [8]:
def get_random_colors():
    colors = webcolors.CSS3_HEX_TO_NAMES
    aborted_colors = ['white','mintcream','snow','lightyellow','whitesmoke','linen','beige','seashell',
                     'floralwhite','oldlace','lavenderblush','ivory','ghostwhite','mediumslateblue',
                     'aliceblue','lightgoldenrodyellow','honeydew','azure','cornsilk','black']
    palette = [colors[key] for key in colors]
    random.shuffle(palette)
    chosen_palette = [palette[i] for i in range(16) if palette[i] not in aborted_colors]
    return chosen_palette

In [9]:
def current_year(year):
    """
    Return: cumulative data for all passed months of the year but with None values for days of future
    months - it plots the data only for passed months.
    """
    wind_grow_ = wind_daily(year, 'all')

    # preparing dataframe for days in future where date is not available yet
    last_date = wind_grow_.index[-1]
    last_day_of_year = datetime(int(year), 12, 31)
    no_data_days = (last_day_of_year - last_date).days # days until the end of year where data not yet available
    no_data_range = pd.date_range(last_date+timedelta(days=1), last_date + timedelta(no_data_days), freq='D')
    data_vals = np.array([None]*no_data_days)
    df = pd.DataFrame({'Date': no_data_range, 'Wind_Daily(MWh)': data_vals})
    df.set_index('Date', inplace=True)
    wind_grow = pd.concat([wind_grow_, df.iloc[:]], axis=0)
    
    return wind_grow

In [10]:
def wind_1(year, month_number=None):
    """
    Input: number of month in range of 1-12 (or number for passed months of present year), by default it is
    random chosen month when function is being called without argument.
    Return: graph presenting daily total wind generation in Poland for given or random chosen month. 
    """
    if incorrect_year(year) is None:
        return
    
    if month_number is None:
        month_number = random.choice(files_list(year)[1][:-1])
    elif month_number not in files_list(year)[1]:
        print("Data for that month is not available or wrong parameter was given for month number.")
        return
    else:
        month_number = month_number
    month = month_name(month_number)    
    wind_m = wind_daily(year, month_number)/10**3
    w_avg = wind_m.iloc[:, 0].mean()
    # preparing plots with plotly + cufflinks
    wind_m.iplot(kind='bar',
                dimensions=[900,450],
                annotations=[dict(
                                x=wind_m.index[-5],
                                y=w_avg,
                                text='Average Power=' + str(round(w_avg, 1)) + ' GWh',
                                textangle=0,
                                showarror=True,
                                arrowhead=17,
                                ax=0,
                                ay=-40)],
                hline=dict(y=wind_m.iloc[:, 0].mean(), color='red', dash='dash'),
                xTitle='Days',
                yTitle='Total Power (GWh)',
                colors=random.choice(get_random_colors()),
                title='Generation of Wind Power for {} of {}'.format(month, year))

In [11]:
def wind_1a(year, month_number=None):
    """
    Input: number of month in range of 1-12 (or number for passed months of present year), by default it is
    random chosen month when function is being called without month argument.
    Return: graph plotted in web browser, presenting daily wind generation in Poland for given or random chosen month. 
    """
    if incorrect_year(year) is None:
        return
    
    if month_number is None:
        month_number = random.choice(files_list(year)[1][:-1])
    elif month_number not in files_list(year)[1]:
        print("Data for that month is not available or wrong parameter was given for month number.")
        return
    else:
        month_number = month_number
    wind_m = wind_daily(year, month_number)/10**3
    month = month_name(month_number)
    m_avg = wind_m.iloc[:,0].mean()
    
    data = [go.Bar(x=wind_m.index, y=wind_m['Wind_Daily(MWh)'].values, marker={'color':'orange'})]      
    layout = {'xaxis':{'title':'Days'}, 'yaxis':{'title':'Total Power (GWh)'},
             'shapes':[{'type':'line', 'x0':wind_m.index[0], 'x1':wind_m.index[-1], 'y0':m_avg, 'y1':m_avg,
                       'line':{'color':'green', 'width':2, 'dash':'longdash'}}],
              'annotations': [{'x': wind_m.index[-3], 'y': m_avg,
                        'text': 'Avg Power=' + str(round(m_avg, 1)) + ' MWh',
                             'showarrow':True, 'arrowhead':1, 'ax':0, 'ay':-30}],
              'autosize':True,
              'title':f'Generation of Wind Power in {month} of {year}'}
    plot(go.Figure(data=data, layout=layout))

In [12]:
def wind_2(year):
    """
    Return: plot presenting wind power generation for each day of given year
    """
    if incorrect_year(year) is None:
        return

    months = 'all' # enables dataframe for all months of year
    wind_y = wind_daily(year, months)/10**3    # to get values in TeraWatt Hours
    # cufflinks style plot - because of interactivity no need to show all dates in x axis
    wind_y.iplot(kind='bar',
                 annotations = [dict(x=wind_y.index[-2],y=wind_y['Wind_Daily(MWh)'].mean(),
                                text='Average Power=' + str(round(wind_y.iloc[:,0].mean(), 1)) + ' GWh',
                               textangle=0, showarrow=True, arrowhead=17, ax=0, ay=-40)],
                 hline=dict(y=wind_y['Wind_Daily(MWh)'].mean(), color='red', dash='dash'),
                 legend='Wind_Daily(GWh)',
                 showlegend=False,
                 xTitle='Days', yTitle='Total Power(GWh)', colors='blue',
                 title="Generation of Wind Power in " + '{}'.format(year), theme='white',
                dimensions=[1000,500])

In [13]:
def wind_3(year):
    """
    Return: plot presenting wind power generation for each month of given year
    """
    if incorrect_year(year) is None:
        return
    
    months = 'all'
    m_names = month_names(year)
    
    # dataframe resampled to months with index being months names
    wind_monthly = wind_daily(year, months).resample('M').sum() / 10**3
    colors = random.choice(get_random_colors())
    data = [go.Bar(x=m_names, y=round(wind_monthly.iloc[:,0], 3), marker=dict(color=colors), name='Power by Month')]
    layout = go.Layout(xaxis=dict(title='Months'), yaxis=dict(title='Total Power (GWh)'),
                       title="Monthly Wind Power Generation in {}".format(year),
                       height=500, width=900,
                       # showlegend=True)
                      )
    iplot(go.Figure(data=data, layout=layout));

In [14]:
def wind_4(year,plot='line'):
    """
    Return: linear or bar graph for cumulative amount of wind energy generated from the beginning of the year
    (some differences in plotting methods - with cufflinks tools and without it).
    Bar chart is returned with any kind of argument put after year number( for example: (2019, 1) or (2018,'bar'))
    """
    if incorrect_year(year) is None:
        return
    
    months = 'all'
    # dataframe resampled to months with index being months names
    w_daily = wind_hourly(year, months).resample('D').sum().iloc[:,[1]]
    w_daily.rename(columns={'Total_Wind_Power(MWh)':'Wind_Daily(MWh)'}, inplace=True)
    
    wind_grow = w_daily.cumsum()/10**3
    last_day = round(wind_grow.max()[0], 1) # total value for the last day of the plot
    if plot=='line':
        return wind_grow.iplot(kind='scatter',
                    width=2,
                    annotations=[dict(
                                x=wind_grow.index[-1],
                                y=last_day,
                                text='Total=' + str(last_day) + ' GWh',
                                textangle=0,
                                showarror=True,
                                arrowhead=1,
                                ax=0,
                                ay=-20)],
                    xTitle='Days', yTitle='Total Power (GWh)', colors='green',
                    title=f"Wind Power Cumulation in {year}", theme='solar', dimensions=(800, 350))   
    else:
        data = [go.Bar(x=wind_grow.index,
                           y=wind_grow['Wind_Daily(MWh)'],
                           name='Total=\n'+str(round(wind_grow.iloc[-1,0], 1)) + ' GWh')]
        layout = go.Layout(xaxis=dict(title='Days'), yaxis=dict(title='Total Power (GWh)'),
                           title="Growth of Wind Power Generation in {}".format(year),
                           height=450, width=900,
                           showlegend=True,
                           legend=dict(x=1.0, y=1.0))
        iplot(go.Figure(data=data, layout=layout))

In [15]:
def wind_4a():
    """
    Return: Separate linear graphs for each year in data folder with cumulative amount of wind energy generated).
    """
    years = years_list()
    
    for year in years:
        wind_4(year)


In [16]:
def wind_4b():
    """
    Return: A single graph of number of plots for every year of wind generation's cumulative value
    """
    data = []
    years = years_list()
    cur_year = years[-1]
    # 2012 year moved to the end of list to facilitate plotting disturbed by the lack of data for first months of 2012
    years.append(years.pop(0))
    for year in years:
        wind_grow = wind_daily(year, 'all')
        # preparing data for current year to facilitate correct plotting
        if year == cur_year:          
            wind_grow = current_year(year)
        
        # code used for leap years(February with 29 days)
        if wind_grow.shape[0] == 366:
            wind_grow.drop(wind_grow.index[59], inplace=True)
        # common adjustments in data for all years
        wind_grow = wind_grow.cumsum()/10**6
        wind_grow.rename(columns={'Wind_Daily(MWh)': year}, inplace=True)
        wind_grow.index = wind_grow.index.strftime('%b %d')
        last_data = round(wind_grow.max()[0], 1)
            
        trace = go.Scatter(x=wind_grow.index, y=wind_grow[year].values,
                           name=f'{year}: {last_data} TWh')
        data.append(trace)
        
    layout = {'xaxis':{'title':'Days of year', 'nticks':25, 'tickangle':-45, 'ticks':'inside'},
              'yaxis':{'title':'Total Power (TWh)'},
             'title':'Cumulative Wind Power Generation in Years',
             'width':900, 'height':550}
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [17]:
def wind_4c():
    """
    Return: Total yearly production of wind energy in GWh.
    """
    years = years_list()
    data = []
    months = 'all'
    for year in years:
        wind_grow = (wind_daily(year, months)/10**6).resample('Y').sum()
        trace = go.Bar(x=wind_grow.index.strftime('%Y'), y=wind_grow['Wind_Daily(MWh)'], name=year)
        data.append(trace)
    layout = {'xaxis':{'title':'Years'}, 'yaxis':{'title':'Total Generation (TWh)'},
             'title':'Total Wind Power Generation in Years',
             'width':900, 'height':550}
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)

In [18]:
def wind_5(year):
    """
    Return: plot showing an average hour wind generation for given year
    """
    df = wind_hourly(year, 'all')
    
    h_wind = df.pivot_table(index='Date', columns='Time', values='Total_Wind_Power(MWh)').mean()
    hour_avg = h_wind.mean()
    
    data = [go.Bar(x=h_wind.index, y=h_wind.values)]
    layout = {'shapes': [{'type': 'line',
                        'x0': h_wind.index[0], 'y0': hour_avg, 'x1': len(h_wind.index), 'y1': hour_avg,
                        'line': {'color': 'red', 'width': 2, 'dash': 'longdash'}}],
            'showlegend':False,
            'annotations': [{'x': h_wind.index[-10], 'y': hour_avg,
                            'text': 'Avg Power=' + str(round(hour_avg, 1)) + ' MWh',
                             'showarrow':True, 'arrowhead':1, 'ax':0, 'ay':-30}],
            'xaxis': {'title': 'Hours'},
            'yaxis': {'title': 'Generation by Hour (MWh)'},
            'title':f"Average Wind Generation per Hour in {year}",
            'width':800, 'height': 400}

    iplot(go.Figure(data=data, layout=layout))

In [19]:
def wind_6(year):
    """
    Return: each month subplots for hour wind generation in given year
    """
    if incorrect_year(year) is None:
        return
    
    rows, cols = (4, 3)
    m_names = [month_name(m_num) for m_num in range(1, 13)]
    fig = subplots.make_subplots(rows=rows, cols=cols,
                            shared_xaxes=True, shared_yaxes=True,
                            subplot_titles=m_names,
                            print_grid=False)
    row, col = 1, 0
    for month_num in files_list(year)[1][:-1]:
        # month dataframe with energy values for each hour
        df = wind_hourly(year, month_num)
        h_wind = df.pivot_table(index='Date', columns='Time', values='Total_Wind_Power(MWh)').mean()
        # average value of wind energy for all day
        day_avg = h_wind.mean()
        m_name = month_name(month_num)
        trace_month_num = go.Bar(x=h_wind.index, y=h_wind.values)
        trace_avg = go.Scatter(
            x=[h_wind.index[0], len(h_wind.index)],
            y=[day_avg]*2,
            mode='lines+text',
            line={'width':0.8},
            text=[None, 'avg=' + str(int(day_avg)) + ' MWh'],
            textposition='middle left')
        if year != 2012:
            if month_num <= (row * cols): col += 1
            else: row += 1; col = 1
        else:
            if month_num <= (row * cols): col += 1
            else: row += 1; col = month_num % cols
        
        fig.append_trace(trace_month_num, row=row, col=col)
        fig.append_trace(trace_avg, row=row, col=col)

    fig.layout.update({'title':'Average Hour Wind Generation in ' + f'{year}',
                       'xaxis':{'title':'Hours'},
                       'yaxis':{'title':'Avg Power'},
                       'showlegend':False,
                       'width':800,
                       'height':700
                       },
                      autosize=True,
                    )
    iplot(fig);    

In [20]:
# returns daily wind generation for given month, if second argument not given month number is taken randomly
wind_1(2016, 2)

In [21]:
# returns graph with wind power generation for given month of year but plotted in web browser
wind_1a(2019, 6)

In [22]:
# returns daily wind generation for given year
wind_2(2016)

In [23]:
# returns separate bar graphs wiht monthly wind power generation for each year
for i in years_list():
    wind_3(i)

In [24]:
# without optional second argument(any kind) linear graph is returned, in other cases function returns bar plot
wind_4(2016, 'akuku')

In [25]:
wind_4(2019)

In [26]:
# returns separate linear graphs with wind power cumulation for each year
wind_4a()

In [27]:
# returns cumulative growth of wind generation for all years
wind_4b()

In [28]:
# returns total wind power generation for all years
wind_4c()

In [29]:
# returns bar plot with average (during 24 hours) wind power generation for given year
wind_5(2015)

In [30]:
# returns average hour wind generation for each month of given year
wind_6(2020)

#### Supplement 
Presenting data in the form of complete dataframe with implementation of tidy approach to structuring data

In [31]:
# preparing the dataframe with daily values for all years
years = years_list()
df_days = wind_hourly(years[1],'all').resample('D').sum().iloc[:,[1]]
df_days.reset_index(inplace=True)
dates = df_days['Date']
dates = dates.apply(lambda x: x.strftime('%d %B'))
df = pd.DataFrame(dates)
df_cum = df.copy()
for year in years:
    if year == '2012':
        df_year = wind_daily(year,'all')
        empty_days = 366 - df_year.shape[0]
        empty_vals = np.array([None]* empty_days)
        date_range = pd.Series(pd.date_range(f'{year}0101', periods=empty_days))
        df_year_ = pd.DataFrame({'Date':date_range, 'Wind_Daily(MWh)':empty_vals})
        df_year_.set_index('Date', inplace=True)
        df_year = pd.concat([df_year_, df_year.iloc[:]], axis=0)
    elif year == years[-1]:
        df_year = current_year(year)
    else:
        df_year = wind_daily(year,'all')
        
    if df_year.shape[0] > 58  and df_year.index[59].day == 29:
        df_year.loc[df_year.index==58] = df_year.iloc[58][0] + df_year.iloc[59][0]
        df_year.drop(df_year.index[59], inplace=True)
    arr = df_year.values
    df[year] = arr

#df.head()

In [32]:
# comparison of monthly wind power values for all years in data
df_total = df.copy()
df_total['Date'] = df_total['Date'].apply(lambda x: datetime.strptime(x,'%d %B'))
df_total.set_index('Date', inplace=True)
df_total = df_total.resample('M').sum()
df_total.reset_index(inplace=True)
df_total['Date'] = df_total['Date'].apply(lambda x: x.strftime('%B'))
# preparing tidy version (long format) of 'df_total' and using plotly.express to visualize data
df_total = pd.melt(df_total, id_vars='Date', var_name='Years', value_name='Monthly Values in TWh')
fig = px.bar(df_total, x='Date', y='Monthly Values in TWh',
             color='Years', barmode='group',
            title=dict(text='Comparison of monthly wind power values for each year', x=0.5, y=0.85),
            labels={'Date':'Months'})
fig.show()

In [33]:
# plot with cumulative wind power for all years - alternative solution for function 'wind_4b'
# prepared from 'df' dataframe (with all daily values), after using melt() function plotly.express library implemented
df_cum = df.copy()
df_cum.iloc[:,1:] = df_cum.iloc[:,1:].cumsum()
#df_cum.head()
df_tidy = df_cum.melt(id_vars='Date', var_name='Years', value_name='Total Power(TWh)')

fig = px.scatter(df_tidy, x='Date', y='Total Power(TWh)', color='Years',
                 title=dict(text='Cumulative Wind Power Generation in Years', x=0.5, y=0.9),
                labels={'Date':'Days of year'})
fig.update_xaxes(nticks=25, tickangle=-45)
fig.update_traces(marker=dict(size=3))
fig.show();

In [57]:
# presentation of total yearly wind generation - alternative to wind_4c() function
years = years_list()
df = pd.DataFrame()
for year in years:
    wind_year = (wind_daily(year,'all')/10**6).resample('Y').sum()
    df[year] = wind_year['Wind_Daily(MWh)'].values
df_tidy = pd.melt(df, var_name='Year', value_name='Total Power')
#df_tidy.head()
fig = px.bar(df_tidy, x='Year', y='Total Power', color='Year',
            labels={'Total Power':'Total Power in TWh'},
            title=dict(text='Wind Energy Total Production per Year', x=0.5, y=0.9))
fig.show()